In [1]:
import pandas as pd
import geopandas as gpd
import os
import numpy as np
import requests
import urllib
import json
import getpass

# funciones desarrolladas
from functions.agrupar_dfs_censo import *
from functions.cargar_data import *
from functions.impresion import *

# Matrices de distancias

## Matriz de distancias entre centros medios de población de cada departamento

In [2]:
# carga capa de centros medios de población
cp = gpd.read_file('capas/centro_poblacion.gpkg')

cp.head()

/home/guillermo/anaconda3/envs/tesis/lib/python3.9/site-packages/geopandas/geodataframe.py:577: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:


,DEPTO,geometry
0,1,POINT (575497.000 6141786.000)
1,2,POINT (517717.000 6636093.000)
2,3,POINT (587353.000 6159278.000)
3,4,POINT (777711.000 6409999.000)
4,5,POINT (431654.000 6211872.000)


In [3]:
def reproyectar_dd(df):
    "Reproyecta coordenadas de DF para usar en Google Distance Matrix API"
    # obtiene coordenadas en EPSG 4326
    lon = df.geometry.to_crs(epsg=4326).x
    lat = df.geometry.to_crs(epsg=4326).y
    df_repro = pd.concat( [ pd.DataFrame(list(lat), columns=['lat'] ), pd.DataFrame( list(lon), columns=['lon'] ) ], axis=1, sort=False)
    # concatena en un sólo string
    df_repro['latlon'] = df_repro.apply(lambda x:'%s,%s' % (x['lat'],x['lon']),axis=1)
    # concatena ambos df
    df_rep_concat = pd.concat([df, df_repro], axis=1, ignore_index=False)
    return df_rep_concat

In [4]:
# aplica función
df_cp = reproyectar_dd(cp)
df_cp.head()

,DEPTO,geometry,lat,lon,latlon
0,1,POINT (575497.000 6141786.000),-34.863473,-56.174030,"-34.86347311448007,-56.17403019571399"
1,2,POINT (517717.000 6636093.000),-30.407062,-56.815547,"-30.407062416732213,-56.81554727891256"
2,3,POINT (587353.000 6159278.000),-34.704806,-56.046153,"-34.70480623405611,-56.04615274218908"
3,4,POINT (777711.000 6409999.000),-32.412531,-54.047130,"-32.412530637702794,-54.04713015664623"
4,5,POINT (431654.000 6211872.000),-34.231989,-57.742109,"-34.23198949912197,-57.74210900659811"


In [5]:
def producto_cartesiano(df, variable):
    "Calcula el producto cartesiano del DF"
    df_coords = df[[variable, 'latlon']]
    df_coords.insert(2, "key", 0, True)
    df_pc = df_coords.merge(df_coords, on='key', how='outer').drop(columns='key')

    dict_rename = {variable + '_x': 'cod_ori',
                variable + '_y': 'cod_des',
                'latlon_x': 'latlon_ori',
                'latlon_y': 'latlon_des'}

    df_pc.rename(columns=dict_rename, inplace=True)

    # elimina misma cod origen-destino
    df_pc = df_pc[df_pc['cod_ori'] != df_pc['cod_des']]

    # concatena en un código de díada
    df_pc['cod'] = (df_pc['cod_ori'].astype(str).str.zfill(2) + 
                    df_pc['cod_des'].astype(str).str.zfill(2)).astype('int')
                    
    return df_pc

In [6]:
df_pc = producto_cartesiano(df_cp, 'DEPTO')

In [7]:
# matriz de distancia entre capitales departamentales de departamentos
def obtenerDist(ori, dest, API_KEY):
    '''
    Devuelve distancia en KM entre uno o varios pares de coordenadas
    geográficas, utilizando la API de Google Distance Matrix.
    '''
    APIurl = """https://maps.googleapis.com/maps/api/distancematrix/json?
                units=metric&mode=walking&origins={}
                &destinations={}&key={}""".format(ori, dest, API_KEY)
    
    # Obtener la respuesta y cargar la representacion en un string
    response = requests.get(APIurl)
    geodataString = response.text
    geodata = json.loads(geodataString)
    # si la API no devulve resultados, la distancia será 'None'
    if geodata['rows'][0]['elements'][0]['status']=='ZERO_RESULTS':
        distancia = None
    else:
        distancia = geodata['rows'][0]['elements'][0]['distance']['value']
    return distancia

In [15]:
# setea la API KEY
API_KEY = getpass.getpass()

# ordena y resetea el índice
df_pc.sort_values(by=['cod_ori', 'cod_des'], inplace=True)
df_pc.reset_index(drop=True, inplace=True)

print(df_pc.shape[0])
df_pc.head(3)

 ·······································


342


,cod_ori,latlon_ori,cod_des,latlon_des,cod
0,1,"-34.86347311448007,-56.17403019571399",2,"-30.407062416732217,-56.81554727891256",102
1,1,"-34.86347311448007,-56.17403019571399",3,"-34.70480623405611,-56.04615274218908",103
2,1,"-34.86347311448007,-56.17403019571399",4,"-32.412530637702794,-54.04713015664623",104


In [16]:
df_pc.shape[0]

342

In [29]:
# prueba con 1 caso
obtenerDist( df_pc.latlon_ori[4] , df_pc.latlon_des[4] , API_KEY)

198970

In [18]:
# son 342 díadas de centros de población
# for loop para obtener una lista con las distancias entre cp
# aplica la función obtenerDistancia para combinaciones de cp
n_iter = df_pc.shape[0]
distancias = [obtenerDist(df_pc.latlon_ori[x] , df_pc.latlon_des[x] , API_KEY) for x in range(n_iter)]

In [20]:
print(len(distancias))

342


In [28]:
dist_series = pd.Series(distancias)
df_pc['distancia'] = dist_series.values
df_pc.head()

,cod_ori,latlon_ori,cod_des,latlon_des,cod,distancia
0,1,"-34.86347311448007,-56.17403019571399",2,"-30.407062416732217,-56.81554727891256",102,610726
1,1,"-34.86347311448007,-56.17403019571399",3,"-34.70480623405611,-56.04615274218908",103,22544
2,1,"-34.86347311448007,-56.17403019571399",4,"-32.412530637702794,-54.04713015664623",104,393703
3,1,"-34.86347311448007,-56.17403019571399",5,"-34.23198949912197,-57.74210900659811",105,172773
4,1,"-34.86347311448007,-56.17403019571399",6,"-33.29103587835989,-56.27098299307005",106,198970


In [30]:
# guarda
df_pc.to_csv('tablas/df_distancias_centro_poblacion.csv', index=False)

## Matriz de distancias entre localidades

In [8]:
locs = cargar_datos_geo()[2][['CODLOC', 'geometry']]
locs.head()

/home/guillermo/anaconda3/envs/tesis/lib/python3.9/site-packages/geopandas/geodataframe.py:577: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:


,CODLOC,geometry
0,2220,POINT (549273.219 6633915.063)
1,2521,POINT (442454.671 6652273.541)
2,2522,POINT (457921.001 6633361.660)
3,2621,POINT (468458.270 6601361.867)
4,2721,POINT (502880.928 6648162.053)


In [9]:
locs_rep = reproyectar_dd(locs)
locs_rep.head()

,CODLOC,geometry,lat,lon,latlon
0,2220,POINT (549273.219 6633915.063),-30.425838,-56.486918,"-30.425838214197817,-56.48691836714405"
1,2521,POINT (442454.671 6652273.541),-30.259811,-57.598207,"-30.25981089095717,-57.59820712208474"
2,2522,POINT (457921.001 6633361.660),-30.431105,-57.438193,"-30.4311049027931,-57.438192602981864"
3,2621,POINT (468458.270 6601361.867),-30.720183,-57.329438,"-30.720182604693377,-57.32943841744266"
4,2721,POINT (502880.928 6648162.053),-30.298277,-56.970040,"-30.298276594314746,-56.970039630361384"


In [78]:
locs_pc = producto_cartesiano(locs_rep, 'CODLOC').reset_index()
print(locs_pc.shape[0])
print(len(locs_pc.cod.unique()))
locs_pc.tail()

377610
377610


,index,cod_ori,latlon_ori,cod_des,latlon_des,cod
377605,378219,1020,"-34.83459368462574,-56.200727322283996",19967,"-32.9586806453632,-53.63484585782342",102019967
377606,378220,1020,"-34.83459368462574,-56.200727322283996",19968,"-33.253755207984504,-53.903051141953384",102019968
377607,378221,1020,"-34.83459368462574,-56.200727322283996",19969,"-32.802391501575336,-53.414203834448486",102019969
377608,378222,1020,"-34.83459368462574,-56.200727322283996",19970,"-33.12425161662233,-54.05902035385788",102019970
377609,378223,1020,"-34.83459368462574,-56.200727322283996",19971,"-32.86300103282358,-53.617689715872686",102019971


In [82]:
# carga distancias entre localidades, precalculadas por pgRouting
distancias_pgr = pd.read_csv('tablas/df_distancias_localidades_pgrouting.csv', sep=';').drop_duplicates('cod')
print(distancias_pgr.shape[0])
print(len(distancias_pgr.cod.unique()))
distancias_pgr.head(2)

376382
376382


,cod,distancia_m
0,10203835,21639
1,10203727,20792


In [83]:
locs_pc_dist = pd.merge(locs_pc, distancias_pgr, how='left', on='cod', indicator=True)
locs_pc_dist.head()

,index,cod_ori,latlon_ori,cod_des,latlon_des,cod,distancia_m,_merge
0,1,2220,"-30.425838214197817,-56.48691836714405",2521,"-30.25981089095717,-57.59820712208474",22202521,137927.0,both
1,2,2220,"-30.425838214197817,-56.48691836714405",2522,"-30.4311049027931,-57.438192602981864",22202522,111408.0,both
2,3,2220,"-30.425838214197817,-56.48691836714405",2621,"-30.720182604693377,-57.32943841744266",22202621,106867.0,both
3,4,2220,"-30.425838214197817,-56.48691836714405",2721,"-30.298276594314746,-56.970039630361384",22202721,58392.0,both
4,5,2220,"-30.425838214197817,-56.48691836714405",2724,"-30.212907501181586,-57.60354994479026",22202724,142223.0,both


In [84]:
locs_pc_dist.groupby('_merge').size()

_merge
left_only       1228
right_only         0
both          376382
dtype: int64

In [85]:
376382 + 1228

377610

In [105]:
locs_pc_dist.shape

(377610, 8)

In [104]:
locs_pc_dist.head()

,index,cod_ori,latlon_ori,cod_des,latlon_des,cod,distancia_m,_merge
0,1,2220,"-30.425838214197817,-56.48691836714405",2521,"-30.25981089095717,-57.59820712208474",22202521,137927.0,both
1,2,2220,"-30.425838214197817,-56.48691836714405",2522,"-30.4311049027931,-57.438192602981864",22202522,111408.0,both
2,3,2220,"-30.425838214197817,-56.48691836714405",2621,"-30.720182604693377,-57.32943841744266",22202621,106867.0,both
3,4,2220,"-30.425838214197817,-56.48691836714405",2721,"-30.298276594314746,-56.970039630361384",22202721,58392.0,both
4,5,2220,"-30.425838214197817,-56.48691836714405",2724,"-30.212907501181586,-57.60354994479026",22202724,142223.0,both


In [15]:
variables = ['cod_ori', 'latlon_ori', 'cod_des', 'latlon_des', 'cod']
locs_pc_sin_dist = locs_pc.loc[locs_pc._merge=='left_only', variables]
locs_pc_sin_dist.head()

,cod_ori,latlon_ori,cod_des,latlon_des,cod
605,2220,"-30.425838214197817,-56.48691836714405",19964,"-32.831506840233985,-53.38397111012659",222019964
1219,2521,"-30.25981089095717,-57.59820712208474",19964,"-32.831506840233985,-53.38397111012659",252119964
1833,2522,"-30.4311049027931,-57.438192602981864",19964,"-32.831506840233985,-53.38397111012659",252219964
2447,2621,"-30.720182604693377,-57.32943841744266",19964,"-32.831506840233985,-53.38397111012659",262119964
3061,2721,"-30.298276594314746,-56.970039630361384",19964,"-32.831506840233985,-53.38397111012659",272119964


In [13]:
# setea la API KEY
API_KEY = getpass.getpass()

In [16]:
# ordena y resetea el índice
locs_pc_sin_dist.sort_values(by=['cod_ori', 'cod_des'], inplace=True)
locs_pc_sin_dist.reset_index(drop=True, inplace=True)

print(locs_pc_sin_dist.shape[0])
locs_pc_sin_dist.head(3)

1228


,cod_ori,latlon_ori,cod_des,latlon_des,cod
0,1020,"-34.83459368462574,-56.200727322283996",19964,"-32.831506840233985,-53.38397111012659",102019964
1,2220,"-30.425838214197817,-56.48691836714405",19964,"-32.831506840233985,-53.38397111012659",222019964
2,2521,"-30.25981089095717,-57.59820712208474",19964,"-32.831506840233985,-53.38397111012659",252119964


In [19]:
376995 + 1228

378223

In [90]:
lista_ori = list(locs_pc_sin_dist.latlon_ori)
lista_des = list(locs_pc_sin_dist.latlon_des)

In [91]:
len(lista_des)

1228

In [92]:
n = locs_pc_sin_dist.shape[0]
# distancias = [obtenerDist(lista_ori[x], lista_des[x] , API_KEY) for x in range(n)]

In [95]:
# guarda lo calculado para no volver a usar la API de Google
with open('tablas/dist_loc_faltantes_google.txt', 'w') as f:
    for item in distancias:
        f.write("%s\n" % item)

In [96]:
locs_pc_sin_dist['distancia_m'] = distancias

In [119]:
locs_dist = locs_pc_dist.loc[locs_pc_dist._merge == 'both'].append(locs_pc_sin_dist).drop(['index','_merge'], axis=1)

In [123]:
locs_dist['distancia_m'] = locs_dist['distancia_m'].astype(int)

In [124]:
locs_dist.head()

,cod_ori,latlon_ori,cod_des,latlon_des,cod,distancia_m
0,2220,"-30.425838214197817,-56.48691836714405",2521,"-30.25981089095717,-57.59820712208474",22202521,137927
1,2220,"-30.425838214197817,-56.48691836714405",2522,"-30.4311049027931,-57.438192602981864",22202522,111408
2,2220,"-30.425838214197817,-56.48691836714405",2621,"-30.720182604693377,-57.32943841744266",22202621,106867
3,2220,"-30.425838214197817,-56.48691836714405",2721,"-30.298276594314746,-56.970039630361384",22202721,58392
4,2220,"-30.425838214197817,-56.48691836714405",2724,"-30.212907501181586,-57.60354994479026",22202724,142223


In [125]:
# guarda
locs_dist.to_csv('tablas/df_distancias_localidades.csv', index=False)

In [1]:
locs_dist.shape

NameError: name 'locs_dist' is not defined